# Vectorization of Statement (문장의 vector 화)

- BOW (Bag of Words)
- TF-IDF (Term Frequency - Inverse Document Frequency)  
- Word Embedding - Keras word API 사용

In [ ]:
import sklearn
sklearn.__version__

'1.2.2'

In [ ]:
import pandas as pd

sentences = ['I love my dog.',
             'I love my cat.',
             'I love my dog and love my cat',
             'You love my dog!',
             'Do you think my dog is amazing?']

## 1. Bag of Word (BOW)

- CountVectorizer
    - min_df : vocabulary 에 포함할 최소 발생 빈도
    - ngram_range : (1, 1) - unigram only, (1, 2) - unigram + bigram
    - max_features : top max_features 만으로 vocabulary 구성
    - token_pattern = (?u)\\b\\w\\w+\\b : unocode 영수자 2 글자 이상만 포함

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer() # 문장을 단어사전으로 만들고 카운트
count_vectorizer

CountVectorizer()

### Text vs token Matrix 생성

In [ ]:
feature = count_vectorizer.fit_transform(sentences) # 단어사전 만듦
feature.shape # (문장수, 어휘수)

(5, 10)

In [ ]:
# 문장 별 단어 빈도 행렬 출력
vectorizerd_sectences = feature.toarray()
vectorizerd_sectences

array([[0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 2, 2, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 1, 1, 1]])

In [ ]:
# features의 단어 리스트
feature_names = count_vectorizer.get_feature_names_out()
feature_names # 단어사전 단어 목록들이 배열로 출력

array(['amazing', 'and', 'cat', 'do', 'dog', 'is', 'love', 'my', 'think',
       'you'], dtype=object)

In [ ]:
# 데이터 프레임으로 변환
df = pd.DataFrame(vectorizerd_sectences, columns=feature_names)
df.index.name = 'sentences' # 인덱스 컬럼 이름 = 'sentence'
df

,amazing,and,cat,do,dog,is,love,my,think,you
sentences,,,,,,,,,,
0,0,0,0,0,1,0,1,1,0,0
1,0,0,1,0,0,0,1,1,0,0
2,0,1,1,0,1,0,2,2,0,0
3,0,0,0,0,1,0,1,1,0,1
4,1,0,0,1,1,1,0,1,1,1


## 2. TF-IDF
- TF-IDF(Term Frequency - Inverse Document Frequency)
- 특정단어가 특정 문서에 출현한 빈도 / 특정 단어가 전체 문서에 출현한 빈도
- 높을수록 그 단어가 중요하다고 간주됨

In [ ]:
from sklearn.feature_extraction.text import  TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer

TfidfVectorizer()

In [ ]:
tfidf_sentences = tfidf_vectorizer.fit_transform(sentences)
tfidf_sentences

<5x10 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [ ]:
# saprse matrix > numpy array로 변환
tfidf_vect_sentences = tfidf_sentences.toarray()
tfidf_vect_sentences

array([[0.        , 0.        , 0.        , 0.        , 0.60685614,
        0.        , 0.60685614, 0.51327503, 0.        , 0.        ],
       [0.        , 0.        , 0.73792244, 0.        , 0.        ,
        0.        , 0.51528988, 0.43582888, 0.        , 0.        ],
       [0.        , 0.49110884, 0.39622352, 0.        , 0.27668216,
        0.        , 0.55336431, 0.46803199, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.45805379,
        0.        , 0.45805379, 0.38741896, 0.        , 0.65595732],
       [0.43872423, 0.        , 0.        , 0.43872423, 0.24716958,
        0.43872423, 0.        , 0.20905445, 0.43872423, 0.35395995]])

In [ ]:
# 위 배열의 컬럼 명 출력

tfidf_feature_names = count_vectorizer.get_feature_names_out()
tfidf_feature_names

array(['amazing', 'and', 'cat', 'do', 'dog', 'is', 'love', 'my', 'think',
       'you'], dtype=object)

In [ ]:
# 배열 > 데이터 프레임으로 변환
df = pd.DataFrame(tfidf_vect_sentences, columns=tfidf_feature_names)
df

,amazing,and,cat,do,dog,is,love,my,think,you
0,0.000000,0.000000,0.000000,0.000000,0.606856,0.000000,0.606856,0.513275,0.000000,0.000000
1,0.000000,0.000000,0.737922,0.000000,0.000000,0.000000,0.515290,0.435829,0.000000,0.000000
2,0.000000,0.491109,0.396224,0.000000,0.276682,0.000000,0.553364,0.468032,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.458054,0.000000,0.458054,0.387419,0.000000,0.655957
4,0.438724,0.000000,0.000000,0.438724,0.247170,0.438724,0.000000,0.209054,0.438724,0.353960


# 3. keras word encoding

- keras  API 이용

In [ ]:
sentences

['I love my dog.',
 'I love my cat.',
 'I love my dog and love my cat',
 'You love my dog!',
 'Do you think my dog is amazing?']

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## Word Index Vocabulary 작성

In [ ]:
# 토크나이저 생성
# num_words : 단어 사전의 단어 개수 제한
# oov_token : 단어 사전에 없는 단어의 처리

tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')

In [ ]:
# 'sentence'에 대해 단어사전 생성 & 정수 부여
tokenizer.fit_on_texts(sentences)

# word_index : 단어 사전을 {단어 : 정수} 형태로 보여줌
word_index = tokenizer.word_index
word_index

{'<OOV>': 1,
 'my': 2,
 'love': 3,
 'dog': 4,
 'i': 5,
 'cat': 6,
 'you': 7,
 'and': 8,
 'do': 9,
 'think': 10,
 'is': 11,
 'amazing': 12}

## text 의 sentence 변환 및 paddding

- **`texts_to_sequences`**
  - 문장(string) 내의 각 단어를 단어사전에 기반하여 수열 (sequence of integers)로 변환
  - 입력 : text (strings) list, 반환 : sequence(정수) list   
- **`pad_sequences`**
  - 동일한 길이로 sequence 를 zero padding
  - 배치처리를 위함임
  - padding : 패딩 문자(0) 채울 위치 지정. 기본값 pre(문장의 앞)
  - truncating : 문장의 길이가 max_len보다 길 경우 자를 위치 지정. 기본값 pre


In [ ]:
# string 타입 문장을 정수형 시퀀스로 변환하여 sequence 변수에 할당
sequences = tokenizer.texts_to_sequences(sentences)

# 시퀀스 배치 처리를 위해 같은 길이로 맞춰줘야 함
# padding : 패딩 문자(0) 채울 위치 지정. 기본값 pre(문장의 앞)
# truncating : 문장의 길이가 max_len보다 길 경우 자를 위치 지정. 기본값 pre
padded = pad_sequences(sequences, padding='post', truncating='post')

In [ ]:
print(sequences)
print(padded) # 패딩처리된 시퀀스

[[5, 3, 2, 4], [5, 3, 2, 6], [5, 3, 2, 4, 8, 3, 2, 6], [7, 3, 2, 4], [9, 7, 10, 2, 4, 11, 12]]
[[ 5  3  2  4  0  0  0  0]
 [ 5  3  2  6  0  0  0  0]
 [ 5  3  2  4  8  3  2  6]
 [ 7  3  2  4  0  0  0  0]
 [ 9  7 10  2  4 11 12  0]]


In [ ]:
# index_word : 단어 사전을 {정수 : 단어} 형태로 보여줌
tokenizer.index_word

{1: '<OOV>',
 2: 'my',
 3: 'love',
 4: 'dog',
 5: 'i',
 6: 'cat',
 7: 'you',
 8: 'and',
 9: 'do',
 10: 'think',
 11: 'is',
 12: 'amazing'}

### sequenced sentence 를 word sentence 로 환원

In [ ]:
# sequece : corpus 내 한 문장

for sequence in sequences:
  sent = []

# 문장에 대해 단어 단위로 순회하며, 정수를 단어로 변환
  for idx in sequence:
    sent.append(tokenizer.index_word[idx]) # 변환한 것을 sent에 저장
  print(' '.join(sent)) # 한 문장에 대해 변환이 완료되면 공백 기준으로 리스트를 뭉침

i love my dog
i love my cat
i love my dog and love my cat
you love my dog
do you think my dog is amazing


### One-Hot-Encoding 표현

In [ ]:
# 패딩 처리된 정수시퀀스를 원핫인코딩
to_categorical(padded)

array([[[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

    